In [2]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [3]:
from attack_probability_evaluation import expectation_of_waiting_time_in_seconds, probability_of_attack

In [4]:
frame_rate = 10 # frames per second

In [5]:
sucess_attack_duration = 1 # seconds
num_bits = 2
num_symbols = 2**num_bits

In [6]:
expectation_of_waiting_time_in_seconds(num_symbols=num_symbols,success_attack_duration_in_seconds=sucess_attack_duration, frame_rate_in_hz=frame_rate)

139810.0

In [7]:
total_time = 120
probability_of_attack(num_symbols=num_symbols, success_attack_duration_in_seconds=sucess_attack_duration, total_time_in_seconds=total_time, frame_rate_in_hz=frame_rate)

0.0008517514268909787

In [32]:
fig = go.Figure()

frame_rate_in_hz = 20
total_time_vec = np.hstack((np.array([0.5]), np.linspace(1, 10, 10), np.array([15, 20, 50, 80, 100])))


colors = ['blue', 'red', 'green']
styles = ['solid', 'dash']

for color, num_symbols in zip(colors, [2, 4, 8]):
    for line_style, success_attack_duration_in_seconds in zip(styles, [0.25, 0.5]):
        total_time_vec_temp = total_time_vec[np.where(total_time_vec >= success_attack_duration_in_seconds)]

        probability_of_attack_vec = np.array([
            probability_of_attack(num_symbols=num_symbols, success_attack_duration_in_seconds=success_attack_duration_in_seconds,
            total_time_in_seconds=total_time, frame_rate_in_hz=frame_rate_in_hz) for total_time in total_time_vec_temp])

        fig.add_trace(go.Scatter(x=total_time_vec_temp, y=probability_of_attack_vec, 
                                mode='lines',
                                name=f'{num_symbols}\t|\t{success_attack_duration_in_seconds} seconds',
                                line=dict(dash=line_style, color=color)))

fig.update_layout(
    yaxis_type="log",
    title=f"Attack Probability V.S Attack Duration for frame rate {frame_rate_in_hz} Hz",
    xaxis_title="Total Attack Duration [seconds]",
    yaxis_title="Successful Attack Probability (Log Scale)",
    xaxis=dict(
        showgrid=True,  # Show grid lines on the x-axis
        gridcolor='LightGray',  # Grid line color
        gridwidth=1  # Grid line width
    ),
    yaxis=dict(
        showgrid=True,  # Show grid lines on the y-axis
        gridcolor='LightGray',  # Grid line color
        gridwidth=1,  # Grid line width
         minor=dict(
            ticklen=4,
            tickcolor='rgba(0,0,0,0.5)',
            gridcolor='LightGray',
            nticks=5  # Number of minor ticks
        ),
    ),
    autosize=False,  # Disable autosizing
    width=1000,       # Set width to a fixed value
    height=600,      # Set height to the same value to make it square,
    legend=dict(
        x=0.83,  # X coordinate in normalized axis coordinates (0-1 range)
        y=0.18,  # Y coordinate in normalized axis coordinates (0-1 range)
        xanchor='center',  # X anchor point for legend box
        yanchor='middle',  # Y anchor point for legend box
        bgcolor='rgba(255, 255, 255, 0.5)',  # Background color with transparency
        bordercolor='Black',  # Border color
        borderwidth=1,  # Border width,
        font=dict(
            size=10,
            color="black"
        ),
        title = '# Used Widths | Minimal Injection Duration',
    )
    
    )
fig.show()

In [42]:
success_attack_duration_in_seconds_vec * frame_rate_in_hz

array([ 2.        ,  2.42105263,  2.84210526,  3.26315789,  3.68421053,
        4.10526316,  4.52631579,  4.94736842,  5.36842105,  5.78947368,
        6.21052632,  6.63157895,  7.05263158,  7.47368421,  7.89473684,
        8.31578947,  8.73684211,  9.15789474,  9.57894737, 10.        ])

In [57]:
np.ceil( * frame_rate_in_hz).astype(int)

array([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21])

In [94]:
fig = go.Figure()

frame_rate_in_hz = 20
success_attack_duration_in_seconds_vec = np.arange(0.1, 0.30, 0.025)


colors = ['blue', 'red', 'green']
# styles = ['solid', 'dash']

for color, num_symbols in zip(colors, [2, 4, 8]):
    
    waiting_time_vec_vec = np.array([
            expectation_of_waiting_time_in_seconds(num_symbols=num_symbols, success_attack_duration_in_seconds=success_attack_duration_in_seconds,
            frame_rate_in_hz=frame_rate_in_hz) for success_attack_duration_in_seconds in success_attack_duration_in_seconds_vec])

    fig.add_trace(go.Scatter(x=success_attack_duration_in_seconds_vec, y=waiting_time_vec_vec, 
                            mode='lines',
                            name=f'{num_symbols}',
                            line=dict(dash='solid', color=color)))
    
    # fig.add_trace(go.Scatter(x=success_attack_duration_in_seconds_vec, y=1/(1/num_symbols)**(np.ceil(success_attack_duration_in_seconds_vec*frame_rate_in_hz))/frame_rate_in_hz, 
    #                         mode='lines',
    #                         name=f'{int(np.log2(num_symbols))}',
    #                         line=dict(dash='dash', color=color)))

# Custom tick labels for the y-axis
def seconds_to_time_format(seconds):
    if seconds < 60:
        return f'{seconds:.0f} sec'
    elif seconds < 3600:
        return f'{seconds / 60:.0f} min'
    else:
        return f'{seconds / 3600:.0f} hrs'
    

ticks_values = [0.1, 1, 5, 10, 60, 5*60, 30*60, 3600, 3600*5]

fig.update_layout(
    yaxis_type="log",
    title=f"Expected Attack Duration v.s Manipulation Duration for frame rate {frame_rate_in_hz} Hz",
    xaxis_title="Minimal Injection Duration [seconds]",
    yaxis_title="Expected Attack Time [seconds] (Log Scale)",
    yaxis_tickformat='s',
    yaxis_tickvals=ticks_values,  # Adjust based on your data
    yaxis_ticktext=[seconds_to_time_format(sec) for sec in ticks_values],
    
    xaxis=dict(
        showgrid=True,  # Show grid lines on the x-axis
        gridcolor='LightGray',  # Grid line color
        gridwidth=1  # Grid line width
    ),
    yaxis=dict(
        showgrid=True,  # Show grid lines on the y-axis
        gridcolor='LightGray',  # Grid line color
        gridwidth=1,  # Grid line width

    ),
    autosize=False,  # Disable autosizing
    width=1000,       # Set width to a fixed value
    height=600,      # Set height to the same value to make it square
    legend=dict(
        x=0.5,  # X coordinate in normalized axis coordinates (0-1 range)
        y=0.9,  # Y coordinate in normalized axis coordinates (0-1 range)
        xanchor='center',  # X anchor point for legend box
        yanchor='middle',  # Y anchor point for legend box
        bgcolor='rgba(255, 255, 255, 0.5)',  # Background color with transparency
        bordercolor='Black',  # Border color
        borderwidth=1,  # Border width,
        font=dict(
            size=10,
            color="black"
        ),
        title = '# Used Widths',
        traceorder='reversed',
        orientation='h'
        )
    )
fig.show()